# Definition of neuron angle and network phase

Intro to PC space phases

In [ ]:
%matplotlib widget
import numpy as np
from sklearn.decomposition import PCA

from matplotlib import pyplot as plt

from lotr import FIGURES_LOCATION, A_FISH, LotrExperiment
from lotr.pca import pca_and_phase
from lotr.notebook_utils import print_source

from lotr.plotting import add_scalebar, add_cbar, color_plot, get_circle_xy, despine, COLS, plot_arrow, \
    get_default_phase_col

In [ ]:
exp = LotrExperiment(A_FISH)

In [ ]:
non_hdns = np.ones(exp.n_rois, dtype=bool)

In [ ]:
t_end = 1500  # crop to half experiment for clarity
pca_scores = PCA(n_components=5).fit_transform(exp.traces[:t_end*exp.fn, exp.hdn_indexes])

In [ ]:
f, ax = plt.subplots(figsize=(3, 3))
s = color_plot(pca_scores[:, 0], pca_scores[:, 1], c=np.arange(pca_scores.shape[0])/exp.fn, ax=ax, 
               cmap=COLS["time"], lw=2)
add_scalebar(ax, xlabel="PC1", ylabel="PC2", xlen=5, ylen=5)

add_cbar(s, ax, inset_loc=(0.85, 0.95, .2, .04), ticks=(0, t_end-100), ticklabels=(0, t_end), 
         orientation="horizontal", label="Time (s)", labelsize=8)
ax.axis("equal")
plt.show()

f.savefig(FIGURES_LOCATION / "population_trajectory.pdf")

## PCA over time and neuron angles

If we now look at PCs computed over time, we get a similar circular pattern:

In [ ]:
# Find principal components from selected neurons:
pca = PCA(n_components=5).fit(exp.traces[:, exp.hdn_indexes].T)

# Compute scores for all neurons:
pca_scores_t = pca.transform(exp.traces[:, :].T)

In [ ]:
f, ax = plt.subplots(figsize=(3, 3))
s = ax.scatter(pca_scores_t[exp.hdn_indexes, 0], pca_scores_t[exp.hdn_indexes, 1], 
               color=COLS["ring"], label="ring ROIs", lw=0.2)
s = ax.scatter(pca_scores_t[exp.nonhdn_indexes, 0], pca_scores_t[exp.nonhdn_indexes, 1], 
               label="non-ring ROIs", fc="none", ec="k", lw=0.2, zorder=-100)

plt.legend(frameon=False, bbox_to_anchor=(1.1, -0.1, .04, .2), bbox_transform=ax.transAxes,
           markerscale=1, handletextpad=-0.3, fontsize=8)
add_scalebar(ax, xlabel="PC1", ylabel="PC2", xlen=30, ylen=30)
ax.axis("equal")
#plt.tight_layout()
plt.subplots_adjust(bottom=0.2)
plt.show()

f.savefig(FIGURES_LOCATION / "individual_rois_pcs_cnt.pdf")

### Neuron angles

On these data, it is simple to fit a circle and define for every ROI a "PC angle" as the angle from the center of such circle. 
For $i^{th}$ ROI,

$$ 
{PCangle}_i = arctan(\frac{PC1_i - center_x}{PC2_i - center_y}) 
$$

For this, we use the `pca_and_phase` function:

In [ ]:
print_source(pca_and_phase)

In [ ]:
pca_scores_t, roi_pc_angles, _, circle_params = pca_and_phase(exp.traces[:, exp.hdn_indexes].T)

# To simplify plotting, we just center the circle on 0:
pca_scores_t[:, :2] = pca_scores_t[:, :2] - circle_params[:2]

In [ ]:
f, ax = plt.subplots(figsize=(3, 3))
s = ax.scatter(pca_scores_t[:, 0], pca_scores_t[:, 1], 
               c=roi_pc_angles, cmap=COLS["phase_light"], label="ring ROIs", lw=0.2)
ax.plot(*get_circle_xy((0, 0, circle_params[2])), c=(0.2,)*3, lw=1)
add_scalebar(ax, xlabel="PC1", ylabel="PC2", xlen=30, ylen=30)
ax.axis("equal")

add_cbar(s, ax, (0.92, 1.1, .2, .04), ticks=(-np.pi+0.2, 0, np.pi-0.2), ticklabels=("-π", 0, "π"), 
         label="PC angle", orientation="horizontal")
plt.show()

f.savefig(FIGURES_LOCATION / "rois_angles.pdf")

## From neuron angles to network phase

Now, we can start from the fit of neuron angles in PC space to get to a reasonable notion of "phase" in time of the network. At every timepoint, we will take an average vector sum over the neurons of the circle, weighted by the intensity of their fluorescence. At every timepoint, the fluorescence intensity is normalized to have 0 sum. This mean that a neuron can have negative weight and hence negative contribution. Zero-sum normalization help reducing biases that might come from non homogeneous distribution of ROIs along the circle. 

In [ ]:
from lotr.utils import get_vect_angle
from lotr.rpca_calculation import get_normalized_coords 

norm_activity = get_normalized_coords(exp.traces[:, exp.hdn_indexes].T).T
avg_vects = np.einsum("ij,ik->jk", norm_activity.T, pca_scores_t[:, :2])

network_phase = get_vect_angle(avg_vects.T)

In [ ]:
scale_arr = 5000
scale_mn = 60
f_lim = 0.01
f, axs = plt.subplots(1, 3, figsize=(8, 3))
for n, (i, ax) in enumerate(zip([900, 1000, 3400], axs)):
    s = ax.scatter(pca_scores_t[:, 0], pca_scores_t[:, 1], 
                   c=norm_activity[i, :], cmap=COLS["dff_opp"], vmin=-f_lim, vmax=f_lim, lw=0.)
    
    for r in range(len(exp.hdn_indexes)):
        angle, length = roi_pc_angles[r], norm_activity[i, r]*scale_arr
        ax.plot([0, np.cos(angle)*length], [0, np.sin(angle)*length], c=(0.7,)*3, lw=0.2)
    
    angle, length = network_phase[i], scale_mn
    ax.plot([0, np.cos(angle)*length], [0, np.sin(angle)*length], 
            c=get_default_phase_col(network_phase[i]), lw=1)
    
    # timestamp:
    ax.text(-100, 100, f"t = {i / exp.fn} s", fontsize=8)

    if n == 0:
        add_scalebar(ax, xlabel="PC1", ylabel="PC2", xlen=30, ylen=30)
        cbar = add_cbar(s, ax, (1.1, 0.2, .2, .04), ticks=(-f_lim+0.001, f_lim-0.001), 
                 ticklabels=("-", "+"), titlesize=8,  title="ΔF weight", orientation="horizontal")
    else:  # dummy for scaling:
        add_scalebar(ax, xlabel="", ylabel="", xlen=30, ylen=30, line_params=dict(lw=0))
        
    ax.axis("equal")
    
f.savefig(FIGURES_LOCATION / "network_phase_explanation.pdf")

In [ ]:
from lotr.plotting.gifs_gen.network_phase import network_phase_animation

In [ ]:
f = network_phase_animation()
# plt.show()